# 説明

*注意：このColabは、Google Colab Pro/Pro+で使用してください。無料版Colabでは画像生成AIの使用が規制されています。

- これは、[Kosinkadink/ComfyUI-AnimateDiff-Evolved](https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved)を手軽に利用するためのColabです。

- 以下の各セルを順に実行してください。3番目のセルを実行後、表示されるリンクをクリックするとComfyUIが起動します（cloudflareが不安定でURLが開かない場合は4番目の予備のセル（local tunnel）をお試しください）。

*Note: This Colab should be used with Google Colab Pro/Pro+.

- This is a Colab for easy use of [Kosinkadink/ComfyUI-AnimateDiff-Evolved](https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved).

- Run each cell below in order; after the third cell, click on the link that appears to start ComfyUI.

In [ ]:
#@title 環境設定：Environment Setup

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
UPDATE_COMFY_UI = False  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes
  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !pwd
  !echo -= Install custom nodes dependencies =-
  ![ -f "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py" ] && python "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py"


-= Initial setup ComfyUI =-
Cloning into 'ComfyUI'...
remote: Enumerating objects: 8318, done.
remote: Counting objects: 100% (2586/2586), done.
remote: Compressing objects: 100% (338/338), done.
remote: Total 8318 (delta 2368), reused 2280 (delta 2247), pack-reused 5732
Receiving objects: 100% (8318/8318), 3.53 MiB | 8.66 MiB/s, done.
Resolving deltas: 100% (5578/5578), done.
/content/ComfyUI
-= Install dependencies =-
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121, https://download.pytorch.org/whl/cu118, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━

In [ ]:
#@title ダウンロード：Download custom nodes and models

# Custom Nodes
!git clone https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved.git custom_nodes/ComfyUI-AnimateDiff-Evolved
!git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git custom_nodes/ComfyUI-VideoHelperSuite

# Using SD1.5 models

## Pixel Art LoRAs for SD1.5
!wget -c https://civitai.com/api/download/models/216899  -O ./models/loras/8bitdiffuser_64x.safetensors # 8bitdiffuser_64x
!wget -c https://civitai.com/api/download/models/104225  -O ./models/loras/damage_pixel.safetensors # damage_pixel

## Checkpoints, VAE
!wget -c https://civitai.com/api/download/models/93295 -O ./models/checkpoints/AnimeKawa.safetensors #for 8bitdiffuser_64x
!wget -c https://civitai.com/api/download/models/105780 -O ./models/checkpoints/t-anime-v3-pruned.safetensors #for damage_pixel
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/

## MotionModule
!wget -c https://huggingface.co/manshoety/AD_Stabilized_Motion/resolve/main/mm-Stabilized_mid.pth -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/models/

## LCM-Lora for SD1.5; Optional. Generation time halved, quality reduced.
#!wget -c https://huggingface.co/latent-consistency/lcm-lora-sdv1-5/resolve/main/pytorch_lora_weights.safetensors  -O ./models/loras/lcm_sd15_lora.safetensors

# Using SDXL models & HotshotXL

## Checkpoints, VAE
#!wget -c https://huggingface.co/hotshotco/SDXL-512/resolve/main/hsxl_base_1.0.f16.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/hotshotco/SDXL-512/resolve/main/vae/diffusion_pytorch_model.safetensors -P ./models/vae/

## Hotshot-XL MotionModule
#!wget -c https://huggingface.co/hotshotco/Hotshot-XL/resolve/main/hsxl_temporal_layers.safetensors -P ./custom_nodes/ComfyUI-AnimateDiff-Evolved/models/

## PixelArt LoRA for SDXL
#!wget -c https://huggingface.co/nerijs/pixel-art-xl/resolve/main/pixel-art-xl.safetensors -P ./models/loras/

## LCM-Lora for SDXL
#!wget -c https://huggingface.co/latent-consistency/lcm-lora-sdxl/resolve/main/pytorch_lora_weights.safetensors  -O ./models/loras/lcm_SDXL_lora.safetensors


Cloning into 'custom_nodes/ComfyUI-AnimateDiff-Evolved'...
remote: Enumerating objects: 730, done.
remote: Counting objects: 100% (348/348), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 730 (delta 272), reused 242 (delta 203), pack-reused 382
Receiving objects: 100% (730/730), 270.99 KiB | 1.10 MiB/s, done.
Resolving deltas: 100% (466/466), done.
Cloning into 'custom_nodes/ComfyUI-VideoHelperSuite'...
remote: Enumerating objects: 738, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 738 (delta 119), reused 140 (delta 100), pack-reused 547
Receiving objects: 100% (738/738), 185.62 KiB | 974.00 KiB/s, done.
Resolving deltas: 100% (318/318), done.
--2023-11-16 07:27:49--  https://civitai.com/api/download/models/216899
Resolving civitai.com (civitai.com)... 104.18.22.206, 104.18.23.206, 2606:4700::6812:16ce, ...
Connecting to civitai.com (civitai.com)|104.18.22.206|:443... connected.
HTTP request 

In [ ]:
#@title 起動：Run ComfyUI with cloudflared

!wget -nc https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server --disable-xformers

File ‘cloudflared-linux-amd64.deb’ already there; not retrieving.

(Reading database ... 120886 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2023.10.0) over (2023.10.0) ...
Setting up cloudflared (2023.10.0) ...
Processing triggers for man-db (2.10.2-1) ...
** ComfyUI start up time: 2023-11-16 08:25:29.159005

Prestartup times for custom nodes:
   0.0 seconds: /content/ComfyUI/custom_nodes/ComfyUI-Manager

Total VRAM 15102 MB, total RAM 12983 MB
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync
VAE dtype: torch.float32
Using pytorch cross attention
### Loading: ComfyUI-Manager (V1.0.1)
### ComfyUI Revision: 1692 [7114cfec] | Released on '2023-11-15'

Import times for custom nodes:
   0.0 seconds: /content/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus
   0.1 seconds: /content/ComfyUI/custom_nodes/ComfyUI-Manager
   0.2 seconds: /content/ComfyUI/custom_nodes/ComfyUI-AnimateDiff-Evolved
   0.3 s

In [ ]:
#@title 起動（予備）：Run ComfyUI with localtunnel

!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server --disable-xformers